# Exploration of Data Set 3

This is the exploratory notebook for data set 3. Data is taken from [nrg.com](https://www.nrg.com/resources/energy-tools/tracking-the-market.html) and is specifically for the electricity data of actual load, day-ahead market and real-time market of New York city. 

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss, ccf

# Set the default color cycle
mpl.style.use('seaborn-v0_8-notebook')
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color='bgrcmyk')

In [2]:
def get_num_duplicates_and_NaNs(df, keep='first'):
    """
        takes a dataframe,
        prints number of missing values and number of duplicates
        returns the boolean array of duplicates
    """
    print(f'There are {df.isnull().values.sum()} missing values or NaNs in the DataFrame.')
    duplicated_ = df.duplicated(keep=keep)
    print(f'There are {duplicated_.sum()} duplicate rows in DataFrame based on all columns.')
    return duplicated_

def plot_series(df=None, column=None, series=pd.Series([]), 
                label=None, ylabel=None, title=None, start=0, end=None):
    """
    Plots a certain time-series which has either been loaded in a dataframe
    and which constitutes one of its columns or it a custom pandas series 
    created by the user. The user can define either the 'df' and the 'column' 
    or the 'series' and additionally, can also define the 'label', the 
    'ylabel', the 'title', the 'start' and the 'end' of the plot.
    """
    sns.set()
    fig, ax = plt.subplots(figsize=(30, 12))
    ax.set_xlabel('Time', fontsize=16)
    if column:
        ax.plot(df[column][start:end], label=label)
        ax.set_ylabel(ylabel, fontsize=16)
    if series.any():
        ax.plot(series, label=label)
        ax.set_ylabel(ylabel, fontsize=16)
    if label:
        ax.legend(fontsize=16)
    if title:
        ax.set_title(title, fontsize=24)
    ax.grid(True)
    return ax

In [3]:
rawAL = pd.read_csv("../data/dat_set_3/market_data_actual_load.csv")
rawDA = pd.read_csv("../data/dat_set_3/market_data_day_ahead.csv")
rawRT = pd.read_csv("../data/dat_set_3/market_data_real_time.csv")

## Actual Load

Preliminary work on actual load data. Check for
* duplicates
* NaN's

In [4]:
## these are superfluous 
rawAL.drop(['label','type'],inplace=True, axis=1)

In [5]:
rawAL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329074 entries, 0 to 329073
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   time    329074 non-null  object 
 1   value   322541 non-null  float64
dtypes: float64(1), object(1)
memory usage: 5.0+ MB


In [6]:
duplicated_ = get_num_duplicates_and_NaNs(rawAL)

There are 6533 missing values or NaNs in the DataFrame.
There are 34 duplicate rows in DataFrame based on all columns.


In [7]:
## convert the time column to datetime

rawAL['time'] = pd.to_datetime(rawAL['time'], utc=True, infer_datetime_format=True)

In [8]:
## remove the duplicates

actual_load = rawAL.drop_duplicates(keep='first', subset=['time'])

In [9]:
## interpolate for the NaN's

actual_load.interpolate(method='linear', limit_direction='forward', inplace=True, axis=0)

In [10]:
## final check: duplicates and NaN's should be gone

duplicated_ = get_num_duplicates_and_NaNs(actual_load, False)

There are 0 missing values or NaNs in the DataFrame.
There are 0 duplicate rows in DataFrame based on all columns.


In [11]:
#actual_load.set_index('time', inplace=True)

## day ahead price

In [12]:
rawDA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26328 entries, 0 to 26327
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   26328 non-null  object 
 1   type    26328 non-null  object 
 2   time    26328 non-null  object 
 3   value   26328 non-null  float64
dtypes: float64(1), object(3)
memory usage: 822.9+ KB


In [13]:
rawDA.head(2)

,label,type,time,value
0,Zone J - NEW YORK CITY (DA),price,2020-10-27 00:00:00,18.44
1,Zone J - NEW YORK CITY (DA),price,2020-10-27 01:00:00,17.83


In [14]:
rawDA.drop(['label','type'], inplace=True, axis=1)

In [15]:
rawDA['time'] = pd.to_datetime(rawDA['time'], utc=True, infer_datetime_format=True)

In [16]:
duplicates_ = get_num_duplicates_and_NaNs(rawDA)

There are 0 missing values or NaNs in the DataFrame.
There are 0 duplicate rows in DataFrame based on all columns.


In [17]:
#day_ahead = rawDA.set_index('time')

In [19]:
len(rawDA)

26328

In [21]:
df_final = rawDA.merge(actual_load, on=['time'], how='inner')

In [22]:
len(df_final)

26273

In [23]:
df_final.drop_duplicates(keep='first', subset=['time'])

,time,value_x,value_y
0,2020-10-27 01:00:00+00:00,17.83,4137.0
1,2020-10-27 02:00:00+00:00,16.44,3984.0
2,2020-10-27 03:00:00+00:00,15.45,3895.0
3,2020-10-27 04:00:00+00:00,16.51,3866.0
4,2020-10-27 05:00:00+00:00,17.04,3909.0
...,...,...,...
26268,2023-10-27 11:00:00+00:00,22.72,5755.0
26269,2023-10-27 12:00:00+00:00,21.71,5753.0
26270,2023-10-27 13:00:00+00:00,21.22,5796.0
26271,2023-10-27 14:00:00+00:00,19.30,5783.0


In [25]:
df_final.set_index('time', inplace=True)

In [26]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26273 entries, 2020-10-27 01:00:00+00:00 to 2023-10-27 15:00:00+00:00
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   value_x  26273 non-null  float64
 1   value_y  26273 non-null  float64
dtypes: float64(2)
memory usage: 615.8 KB
